![video_game](video_game.jpg)


Video games are big business: the global gaming market is projected to be worth more than $300 billion by 2027 according to Mordor Intelligence. With so much money at stake, the major game publishers are hugely incentivized to create the next big hit. But are games getting better, or has the golden age of video games already passed?

In this project, you'll analyze video game critic and user scores as well as sales data for the top 400 video games released since 1977. You'll search for a golden age of video games by identifying release years that users and critics liked best, and you'll explore the business side of gaming by looking at game sales data.

Your search will involve joining datasets and comparing results with set theory. You'll also filter, group, and order data. Make sure you brush up on these skills before trying this project! The database contains two tables. Each table has been limited to 400 rows for this project, but you can find the complete dataset with over 13,000 games on Kaggle.

### `game_sales` table

| Column | Definition | Data Type |
|-|-|-|  
|name|Name of the video game|`varchar`|
|platform|Gaming platform|`varchar`|
|publisher|Game publisher|`varchar`|
|developer|Game developer|`varchar`|
|games_sold|Number of copies sold (millions)|`float`|
|year|Release year|`int`|

### `reviews` table

| Column | Definition | Data Type |
|-|-|-|
|name|Name of the video game|`varchar`|  
|critic_score|Critic score according to Metacritic|`float`|
|user_score|User score according to Metacritic|`float`|


### `users_avg_year_rating` table

| Column | Definition | Data Type |
|-|-|-|
|year| Release year of the games reviewed |`int`|  
|num_games| Number of games released that year |`int`|
|avg_user_score| Average score of all the games ratings for the year |`float`|

### `critics_avg_year_rating` table

| Column | Definition | Data Type |
|-|-|-|
|year| Release year of the games reviewed |`int`|  
|num_games| Number of games released that year |`int`|
|avg_critic_score| Average score of all the games ratings for the year |`float`|



In [12]:
-- Top 10 best selling Nintendo games in 2005-2010

SELECT *
FROM game_sales
WHERE year BETWEEN 2005 AND 2010 
	AND publisher = 'Nintendo'
ORDER BY games_sold DESC
LIMIT 10;

In [16]:
-- Top 10 years with the highest average critic score
-- Filtered to only years where at least 5 games were released to ensure a good sample size

SELECT year, 
	   COUNT(g.name) AS num_games, 
	   ROUND(AVG(critic_score),2) AS avg_critic_score
FROM game_sales g
INNER JOIN reviews r
ON g.name = r.name
GROUP BY g.year
HAVING COUNT(g.name) >= 5
ORDER BY avg_critic_score DESC
LIMIT 10;

In [20]:
-- Top 10 games from the year with the highest yearly average critic score
-- Using the previous query as the CTE

WITH highest_reviewed_year AS (
							SELECT g.year, 
	   								COUNT(g.name) AS num_games, 
									ROUND(AVG(critic_score),2) AS avg_critic_score
							FROM game_sales g
							INNER JOIN reviews r
							ON g.name = r.name
							GROUP BY g.year
							HAVING COUNT(g.name) >= 5
							ORDER BY avg_critic_score DESC
							LIMIT 1
) 

SELECT  g.name, r.critic_score 
FROM highest_reviewed_year h
INNER JOIN game_sales g
ON h.year = g.year
INNER JOIN reviews r
ON g.name = r.name
ORDER BY r.critic_score DESC
LIMIT 10;


In [22]:
-- The golden_years of video games where avg critic score and avg user score both exceed rating of 9.0 

SELECT u.year, 
	   u.num_games, 
	   avg_critic_score, 
	   avg_user_score, 
	   abs(avg_critic_score - avg_user_score) AS diff
FROM users_avg_year_rating u
INNER JOIN critics_avg_year_rating c
ON u.year = c.year 
WHERE avg_critic_score > 9 OR avg_user_score > 9
ORDER BY diff;

In [25]:
-- Top 10 video games of all time based on the combination of the critic score and the user score

SELECT g.year, g.name, (r.critic_score + r.user_score)/2 AS avg_score, g.publisher, g.games_sold
FROM game_sales g
INNER JOIN reviews r
ON g.name = r.name
WHERE critic_score IS NOT NULL AND user_score IS NOT NULL
ORDER BY avg_score DESC
LIMIT 10;

,year,name,avg_score,publisher,games_sold
0,2017,The Legend of Zelda: Breath of the Wild for NS,9.95,Nintendo,17.41
1,1998,The Legend of Zelda: Ocarina of Time for N64,9.95,Nintendo,7.60
2,2018,God of War (2018) for PS4,9.85,Sony Interactive Entertainment,11.00
3,2010,Super Mario Galaxy 2 for Wii,9.75,Nintendo,7.41
4,2010,Red Dead Redemption for X360,9.75,Rockstar Games,6.50
5,2007,Super Mario Galaxy for Wii,9.65,Nintendo,12.80
6,2008,Metal Gear Solid 4: Guns of the Patriots for PS3,9.55,Konami,6.00
7,2007,Call of Duty 4: Modern Warfare for PS3,9.55,Activision,6.72
8,1997,Final Fantasy VII for PS,9.55,Sony Computer Entertainment,9.90
9,2007,God of War II for PS2,9.55,Sony Computer Entertainment,4.24
